# Zad 3

In [1]:
import numpy as np
from tqdm import  tqdm_notebook, tnrange

In [2]:
initialize = False

In [3]:
raw_lemmas = []
for line in open('Dane/polimorfologik-2.1.txt', 'r', encoding='utf8'):
    raw_lemmas.append(line.split(';')[:2])

In [4]:
lemmas = {}
inverted_lemmas = {}
for [lemma, word] in raw_lemmas:
    if word in lemmas:
        lemmas[word].append(lemma)
    else:
        lemmas[word] = [lemma]
    if initialize: 
        if lemma in inverted_lemmas:
            inverted_lemmas[lemma].append(word)
        else:
            inverted_lemmas[lemma] = [word]

del raw_lemmas

In [6]:
super_lemmas = {}
if initialize:
    for word in tqdm_notebook(lemmas):
        super_lemmas[word] = lemmas[word]
        for word_lemma in lemmas[word]:
            for inv_lemma in inv_lemmas[word_lemma]:
                super_lemmas[word] = np.unique(np.concatenate((super_lemmas[word], np.array(lemmas[inv_lemma]))))

    del inv_lemmas
    with open('Dane/super_lemmas.txt', 'w', encoding='utf8') as file:
        for line in super_lemmas:
            file.write(line + ' ' + ' '.join(super_lemmas[line]))
            file.write('\n')
else:
    for line in open('Dane/super_lemmas.txt', 'r', encoding='utf8'):
        line = line.split()
        super_lemmas[line[0]] = line[1:]

In [9]:
def get_super_lemma(query):
    query = query.split()
    super_lemma = ''
    for word in query:
        if word not in super_lemmas:
            return False
        super_lemma += '|'.join(super_lemmas[word]) + ' '
    return super_lemma[:-1]

In [10]:
get_super_lemma('piłem wino na barce')

'niepijąca|niepijący|picie|pila|pilić|pita|pić|piła wina|wino|winąć na barek|bark|barka|barki'

## 2-grams

In [11]:
sl_2gram = {}
for line in open('Dane/2grams', 'r', encoding='utf8'):
    gram = line.split(' ')
    if int(gram[-3]) > 100:
        gram = ' '.join(gram[-2:])[:-1]
        sup_lemma = get_super_lemma(gram)
        if not sup_lemma:
            continue
        if sup_lemma not in sl_2gram:
            sl_2gram[sup_lemma] = [gram]
        else:
            sl_2gram[sup_lemma].append(gram)

In [12]:
for key, array in list(sl_2gram.items()):
    if len(array) < 2:
        del sl_2gram[key]

In [13]:
with open('Dane/output_2grams.txt', 'w', encoding='utf8') as file:
    for key, value in sl_2gram.items():
        n = len(value)
        for i in range(n):
            ref_lemmas = [lemmas[tok] for tok in value[i].split()]
            for j in range(i + 1, n):
                comp_lemmas = [lemmas[tok] for tok in value[j].split()]
                fun_words = []
                for k in range(2):
                    if len([l for l in comp_lemmas[k] if l in ref_lemmas[k]]) == 0:
                        fun_words.append(k)
                
                if len(fun_words) != 0:
                    file.write('Superlemma: ' + key + '\n')
                    file.write('First:\n')
                    for k in range(2):
                        if k in fun_words:
                            file.write('- [' + value[i].split()[k] + ']' + ' (' + ', '.join(ref_lemmas[k]) + ') ')
                        else:
                            file.write('- ' + value[i].split()[k] + ' ')
                    file.write('\nSecond:\n')
                    for k in range(2):
                        if k in fun_words:
                            file.write('- [' + value[j].split()[k] + ']' + ' (' + ', '.join(comp_lemmas[k]) + ') ')
                        else:
                            file.write('- ' + value[j].split()[k] + ' ')
                    file.write('\n' + '-' * 30 + '\n')

## 3-grams

In [14]:
sl_3gram = {}
for line in open('Dane/3grams', 'r', encoding='utf8'):
    gram = line.split(' ')
    if int(gram[-4]) > 10:
        gram = ' '.join(gram[-3:])[:-1]
        sup_lemma = get_super_lemma(gram)
        if not sup_lemma: # if invalid token in gram
            continue
        if sup_lemma not in sl_3gram:
            sl_3gram[sup_lemma] = [gram]
        else:
            sl_3gram[sup_lemma].append(gram)

In [15]:
for key, array in list(sl_3gram.items()):
    if len(array) < 2:
        del sl_3gram[key]

In [16]:
with open('Dane/output_3grams.txt', 'w', encoding='utf8') as file:
    for key, key_lemmas in sl_3gram.items():
        n = len(key_lemmas)
        for i in range(n):
            ref_lemmas = [lemmas[tok] for tok in key_lemmas[i].split()]
            for j in range(i + 1, n):
                comp_lemmas = [lemmas[tok] for tok in key_lemmas[j].split()]
                fun_words = []
                for k in range(3):
                    if len([l for l in comp_lemmas[k] if l in ref_lemmas[k]]) == 0:
                        fun_words.append(k)
                
                if len(fun_words) != 0:
                    file.write('Superlemma: ' + key + '\n')
                    file.write('First:\n')
                    for k in range(3):
                        if k in fun_words:
                            file.write('- [' + key_lemmas[i].split()[k] + ']' + ' (' + ', '.join(ref_lemmas[k]) + ') ')
                        else:
                            file.write('- ' + key_lemmas[i].split()[k] + ' ')
                    file.write('\nSecond:\n')
                    for k in range(3):
                        if k in fun_words:
                            file.write('- [' + key_lemmas[j].split()[k] + ']' + ' (' + ', '.join(comp_lemmas[k]) + ') ')
                        else:
                            file.write('- ' + key_lemmas[j].split()[k] + ' ')
                    file.write('\n' + '-' * 30 + '\n')